In [1]:
import gensim
from gensim.models import KeyedVectors
from gensim.models.fasttext import FastText as FT_gensim
import pandas as pd
import numpy as np

In [2]:
# FT_gensim.load('../wordvectors/wiki-news-300d-1M-subword.vec.zip')
wiki_vec = KeyedVectors.load_word2vec_format('../wordvectors/wiki.en.vec', binary=False, limit=10000)

In [3]:
!pwd

/home/ubuntu/code/wikiconflict


In [38]:
wiki_vec["he"].size

300

In [5]:
class Wiki:
    def __init__(self,id,title, all_tokens=[]):
        self.id = id
        self.title = title
        self.all_tokens = all_tokens
        
    def init_revisions(self, revisions):
        self.revisions = {revision["id"] : Revision(revision["id"],revision["timestamp"], revision["editor"]) for revision in revisions}
    
class Revision:
    def __init__(self, id, timestamp,editor):
        self.id = id
        self.timestamp = timestamp
        self.editor = editor
        self.added = set()
        self.removed = set()
        
class Change:
    def __init__(self, token, left_context, right_context):
        self.token = token
        self.from = from
        self.to = to
        self.left = left_context
        self.right = right_context
        
test_wiki = Wiki(2345, "a test wiki", all_tokens=4)
test_wiki.init_revisions([{'timestamp': '2004-04-24T07:40:27Z',
  'id': 3317807,
  'editor': '0|172.194.5.181'},
 {'timestamp': '2004-04-24T07:49:11Z',
  'id': 3317866,
  'editor': '0|172.194.5.181'},
 {'timestamp': '2004-04-24T07:49:11Z',
  'id': 3317921,
  'editor': '0|172.194.5.181'}])
test_wiki.all_tokens = [{'token_id': 0, 'o_rev_id': 3317807, 'str': 'an', 'in': [], 'out': [3317921]},
 {'token_id': 1, 'o_rev_id': 3317807, 'str': '80', 'in': [], 'out': [3317921]}]


In [6]:
for token in test_wiki.all_tokens:
        test_wiki.revisions[token["o_rev_id"]].added.add(token["token_id"])
        for in_revision in token["in"]:
            test_wiki.revisions[in_revision].added.add(token["token_id"])
        for out_revision in token["out"]:
            test_wiki.revisions[out_revision].removed.add(token["token_id"])

In [7]:
test_wiki.revisions[3317921].content = pd.DataFrame( [{'str': 'an', 'token_id': 0},
 {'str': '80', 'token_id': 1},
 {'str': "'", 'token_id': 2},
 {'str': 's', 'token_id': 3},
 {'str': 'u', 'token_id': 4}])

In [8]:
test_wiki.revisions[3317921].content

,str,token_id
0,an,0
1,80,1
2,',2
3,s,3
4,u,4


In [14]:
removed = test_wiki.revisions[3317921].content["token_id"].isin(test_wiki.revisions[3317921].removed)
removed

0     True
1     True
2    False
3    False
4    False
Name: token_id, dtype: bool

In [37]:
np.ediff1d([0,1,0,0,1,1,0,1,1,1])

array([ 1, -1,  0,  1,  0, -1,  1,  0,  0])

In [36]:
np.argwhere(np.ediff1d([0,1,0,0,1,1,0,1,1,1])==-1) +1

array([[2],
       [6]])

In [35]:
np.argwhere(np.ediff1d([0,1,0,0,1,1,0,1,1,1])==1) 

array([[0],
       [3],
       [6]])